In [1]:
import praw
from psaw import PushshiftAPI
import datetime
from datetime import timedelta, date
import pandas as pd 

In [2]:
c_id = ''
c_secret = ''
username = ''
password = ''
targetSubreddit = 'worldnews'
reddit = praw.Reddit(client_id = c_id,
                     client_secret = c_secret,
                     username = username,
                     password = password,
                     user_agent= targetSubreddit
                    )

In [3]:
api = PushshiftAPI(reddit)

In [4]:
start_date = date(2020, 2, 1) #(year, month, day)
end_date = date(2020, 2, 2)
df = pd.DataFrame(columns=['Date', 'Num_comments', 'News'])

In [5]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [6]:
i = 0
for single_date in daterange(start_date, end_date):
    sigle_date = single_date.strftime("%Y-%m-%d")
    year = single_date.year
    month = single_date.month
    day = single_date.day
    start_epoch=int(datetime.datetime(year, month, day, 0, 0, 0).timestamp()) #year, month, day,hour, minutes, seconds
    end_epoch=int(datetime.datetime(year, month, day, 23, 59, 59).timestamp())
    #end_epoch =  dt.datetime.now().timestamp()
    try:
        submissions = list(api.search_submissions(after = start_epoch, before = end_epoch, subreddit = targetSubreddit, 
                                          filter=['title'], limit = 30, sort = 'desc', sort_type = 'num_comments'))
    except:
        continue

    for submission in submissions:
        time = submission.created
        date = datetime.datetime.fromtimestamp(time).strftime("%Y-%m-%d")
        df.loc[i] = [date, submission.num_comments, submission.title]
        i = i + 1
        #print(i)
        # print(submission.num_comments)
        #print(date)
        # print(submission.title)
print("Total: ", i)

Total:  0


#### Sorting the Data and Saving It 

In [7]:
print(df.head(5))
df = df.sort_values(['Date', 'Num_comments'], ascending=[True, False])
#df = df.sort_values(by='Date')
#daily_news = df.groupby("Date")['News'].apply('<.>'.join).reset_index()
print(df.head(4))
#path = '../datasets/subreddit-worldnews-DJIA-dataset/raw-subreddit-worldnews.txt'
#df.to_csv(path, sep='\t', index=False, mode='a', header=False)

Empty DataFrame
Columns: [Date, Num_comments, News]
Index: []
Empty DataFrame
Columns: [Date, Num_comments, News]
Index: []


#### Deleting the 'Num_comments' column and Saving the Modified Data.

In [8]:
#deleting the Num_comments column
path = '../datasets/subreddit-worldnews-DJIA-dataset/raw-subreddit-worldnews.txt'
data = pd.read_csv(path, sep="\t")
print(data.head(5))
del data['Num_comments']
print(data.head(5))
#saving the modified data
path = '../datasets/subreddit-worldnews-DJIA-dataset/subreddit-worldnews.txt'
data.to_csv(path, sep='\t', index=False)

         Date  Num_comments                                               News
0  2008-03-01           105      Israeli minister vows Palestinian 'holocaust'
1  2008-03-01             3          Construction in Michigan Has Gone Too Far
2  2008-03-01             2  Iraq was everyone’s loss, whichever side you w...
3  2008-03-01             1  How the BBC has bid £400m of your money for Eu...
4  2008-03-01             1  Witch convicted of leaking WWII secrets about ...
         Date                                               News
0  2008-03-01      Israeli minister vows Palestinian 'holocaust'
1  2008-03-01          Construction in Michigan Has Gone Too Far
2  2008-03-01  Iraq was everyone’s loss, whichever side you w...
3  2008-03-01  How the BBC has bid £400m of your money for Eu...
4  2008-03-01  Witch convicted of leaking WWII secrets about ...
